In [5]:
m21_y = 14788800
m79_y = 16445000
m_diff = m79_y - m21_y

s21_y = 5569150
s79_y = 7556040
s_diff = s79_y - s21_y

print(f"March ice area difference: {m_diff} km2")
print(f"September ice area difference: {s_diff} km2")

March ice area difference: 1656200 km2
September ice area difference: 1986890 km2


In [8]:
year = (1000000-1.6843*10**8)/-80976.9
print(f"Year when the ice area will be 1 million km2: {year}")
years_left = year - 2024
print(f"Years left until the ice area will be 1 million km2: {years_left}")

Year when the ice area will be 1 million km2: 2067.626693538528
Years left until the ice area will be 1 million km2: 43.62669353852789
